In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import math
from sklearn.preprocessing import StandardScaler

In [2]:
scaler_X=StandardScaler()
#first produce X_train
df_train = pd.read_csv('data/training.csv')
df_test = pd.read_csv('data/testing.csv')
df = pd.read_csv('data/race-result-horse.csv')
X_train=df_train.drop(['finishing_position','horse_number','horse_name','horse_id','jockey','trainer','length_behind_winner','running_position_1','running_position_2','running_position_3','running_position_4','finish_time','running_position_5','running_position_6','race_id','recent_6_runs'],axis=1)
X_train=pd.DataFrame(X_train,dtype=np.float)
scaler_X.fit(X_train)
X_train_normalized=scaler_X.transform(X_train)
m_train, _ = np.shape(X_train)
X_test=df_test.drop(['finishing_position','horse_number','horse_name','horse_id','jockey','trainer','length_behind_winner','running_position_1','running_position_2','running_position_3','running_position_4','finish_time','running_position_5','running_position_6','race_id','recent_6_runs'],axis=1)
X_test=pd.DataFrame(X_test,dtype=np.float)
scaler_X.fit(X_test)
X_test_normalized=scaler_X.transform(X_test)
m_test, _ = np.shape(X_test)
y_train=df_train.finish_time
def t2s(t):#transfer the finish_time into float
    min,sec,msec=t.strip().split(".")
    return (float(min)*60+float(sec)+float(msec)*0.01)
y_train=[t2s(y_train[i]) for i in range(len(y_train))]
y_train=pd.DataFrame(y_train,dtype=np.float)
scaler_y=StandardScaler()
scaler_y.fit(y_train)
y_train_normalized=scaler_y.transform(y_train)
y_test=df_test.finish_time
y_test=[t2s(y_test[i]) for i in range(len(y_test))]
y_test=pd.DataFrame(y_test,dtype=np.float)
scaler_y.fit(y_test)
y_test_normalized=scaler_y.transform(y_test)
m, n = np.shape(df)
m_train, _ = np.shape(df_train)
m_test, _ = np.shape(df_test)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:30: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:33: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:50: FutureWarning: reshape is

In [3]:
#4.1.1
from sklearn.svm import SVR
svr_model=SVR(C=27,epsilon=0.2)
svr_model.fit(X_train,y_train)
svr_result=svr_model.predict(X_test)
svr_score=svr_model.score(X_test,y_test)
print(svr_score)

,finishing_position,horse_number,horse_name,horse_id,jockey,trainer,actual_weight,declared_horse_weight,draw,length_behind_winner,...,finish_time,win_odds,running_position_5,running_position_6,race_id,recent_6_runs,recent_ave_rank,jockey_ave_rank,trainer_ave_rank,race_distance
0,1,1.0,DOUBLE DRAGON,K019,B Prebble,D Cruz,133,1032,1,-,...,1.22.33,3.8,NaN,NaN,2014-001,NaN,7.00,7,7,1400
1,2,2.0,PLAIN BLUE BANNER,S070,D Whyte,D E Ferraris,133,1075,13,2,...,1.22.65,8.0,NaN,NaN,2014-001,NaN,7.00,7,7,1400
2,3,10.0,GOLDWEAVER,P072,Y T Cheng,Y S Tsui,121,1065,3,2,...,1.22.66,5.7,NaN,NaN,2014-001,NaN,7.00,7,7,1400
3,4,3.0,SUPREME PROFIT,P230,J Moreira,C S Shum,132,1222,2,2,...,1.22.66,6.1,NaN,NaN,2014-001,NaN,7.00,7,7,1400
4,5,7.0,THE ONLY KID,H173,Z Purton,K W Lui,125,1136,9,4-1/4,...,1.23.02,6.1,NaN,NaN,2014-001,NaN,7.00,7,7,1400
5,6,9.0,WINNING ADVANTAGE,N359,A Suborics,A T Millard,123,1100,11,5-1/2,...,1.23.20,24.0,NaN,NaN,2014-001,NaN,7.00,7,7,1400
6,7,13.0,CARE FREE ELEGANCE,P340,C Y Ho,K L Man,115,1053,12,5-1/2,...,1.23.22,99.0,NaN,NaN,2014-001,NaN,7.00,7,7,1400
7,8,4.0,COOL PAL,S035,H W Lai,L Ho,129,1203,8,5-3/4,...,1.23.25,21.0,NaN,NaN,2014-001,NaN,7.00,7,7,1400
8,9,6.0,TAI PO FORTUNE,P081,K Teetan,T P Yung,127,1073,6,6-1/4,...,1.23.33,10.0,NaN,NaN,2014-001,NaN,7.00,7,7,1400
9,10,11.0,SUPER HORSE,L308,T H So,C W Chang,119,1137,7,6-3/4,...,1.23.41,27.0,NaN,NaN,2014-001,NaN,7.00,7,7,1400


In [ ]:
#4.1.2
from sklearn.ensemble import GradientBoostingRegressor
gbrt_model=GradientBoostingRegressor(loss='huber',learning_rate=0.05,n_estimators=300,max_depth=1)
gbrt_model.fit(X_train,y_train)
gbrt_result=gbrt_model.predict(X_test)
gbrt_score=gbrt_model.score(X_test,y_test)
print(gbrt_score)

In [ ]:
#4.2
#root_mean_squared_error svr model
y_subtracted=np.power(np.array(y_test)-np.array(svr_result),2)
sum=0
for i in range(len(y_test)):
    sum=sum+y_subtracted[i][0]
root_mean_squared_error_SVR = np.power(sum/len(y_test),1/2)
#root_mean_squared_error GBRT model
y_subtracted=np.power(np.array(y_test)-np.array(gbrt_result),2)
sum=0
for i in range(len(y_test)):
    sum=sum+y_subtracted[i][0]
root_mean_squared_error_GBRT= np.power(sum/len(y_test),1/2)
#get rank of gbrt model from finish_time
raceid=list()
for i in range(len(df_test)):
    if df_test.race_id[i] not in raceid:
        raceid.append(df_test.race_id[i])
#svr
svr_predicted_rank=np.zeros((len(X_test),1))
for i in range(len(raceid)):
    index=list()
    for j in np.where(df_test.race_id==raceid[i]):
        index.append(j)#index is the row number in X_test whose race id is race_id[i]
    index=index[0]
    order=np.argsort(np.array(svr_result)[index],axis=0)
    k=0
    for j in index:
        svr_predicted_rank[j]=order[k]+1
        k=k+1
from collections import Counter
TP_top1=0
TP_top3=0
sum_of_actual_rank=0
count_of_1=0
for i in range(len(svr_result)):
    if svr_predicted_rank[i]==1:
        count_of_1=count_of_1+1
        sum_of_actual_rank=sum_of_actual_rank+df_test.finishing_position[i]
    if (svr_predicted_rank[i]==1 and df_test.finishing_position[i]==1):
        TP_top1=TP_top1+1
    if (svr_predicted_rank[i]==1 and df_test.finishing_position[i]<=3):
        TP_top3=TP_top3+1
Top_1=TP_top1/count_of_1
Top_3=TP_top3/count_of_1
ave_rank=sum_of_actual_rank/count_of_1
print("SVR Model: RMSE = ",root_mean_squared_error_SVR,"; Top_1 = ",Top_1,"; Top_3 = ",Top_3,"; Average_Rank = ",ave_rank)
#gbrt
gbrt_predicted_rank=np.zeros((len(X_test),1))
for i in range(len(raceid)):
    index=list()
    for j in np.where(df_test.race_id==raceid[i]):
        index.append(j)#index is the row number in X_test whose race id is race_id[i]
    index=index[0]
    order=np.argsort(np.array(gbrt_result)[index],axis=0)
    k=0
    for j in index:
        gbrt_predicted_rank[j]=order[k]+1
        k=k+1
from collections import Counter
TP_top1=0
TP_top3=0
sum_of_actual_rank=0
count_of_1=0
for i in range(len(gbrt_result)):
    if gbrt_predicted_rank[i]==1:
        count_of_1=count_of_1+1
        sum_of_actual_rank=sum_of_actual_rank+df_test.finishing_position[i]
    if (gbrt_predicted_rank[i]==1 and df_test.finishing_position[i]==1):
        TP_top1=TP_top1+1
    if (gbrt_predicted_rank[i]==1 and df_test.finishing_position[i]<=3):
        TP_top3=TP_top3+1
Top_1=TP_top1/count_of_1
Top_3=TP_top3/count_of_1
ave_rank=sum_of_actual_rank/count_of_1
print("Gradient Boosting Regression Tree Model: RMSE = ",root_mean_squared_error_GBRT,"; Top_1 = ",Top_1,"; Top_3 = ",Top_3,"; Average_Rank = ",ave_rank)


In [ ]:
#normalized cases
#normalized svr
svr_normalized_model=SVR(C=27,epsilon=0.2)
svr_normalized_model.fit(X_train_normalized,y_train_normalized)
svr_normalized_result=svr_normalized_model.predict(X_test_normalized)
svr_normalized_score=svr_normalized_model.score(X_test_normalized,y_test_normalized)
print(svr_normalized_score)
#normalized GBRT
gbrt_normalized_model=GradientBoostingRegressor(loss='ls',learning_rate=0.01,n_estimators=500,max_depth=1)#very sensitive to leaning_rate
gbrt_normalized_model.fit(X_train_normalized,y_train_normalized)
gbrt_normalized_result=gbrt_normalized_model.predict(X_test_normalized)
gbrt_normalized_score=gbrt_normalized_model.score(X_test_normalized,y_test_normalized)
print(gbrt_normalized_score)
#root_mean_squared_error svr model
y_subtracted=np.power(np.array(y_test_normalized)-np.array(svr_normalized_result),2)
sum=0
for i in range(len(y_test)):
    sum=sum+y_subtracted[i][0]
normalized_root_mean_squared_error_SVR = np.power(sum/len(y_test),1/2)
print(normalized_root_mean_squared_error_SVR)
#root_mean_squared_error GBRT model
y_subtracted=np.power(np.array(y_test_normalized)-np.array(gbrt_normalized_result),2)
sum=0
for i in range(len(y_test)):
    sum=sum+y_subtracted[i][0]
normalized_root_mean_squared_error_GBRT= np.power(sum/len(y_test),1/2)
print(normalized_root_mean_squared_error_GBRT)
